In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

import cobra
from cameo import pfba
from cameo.flux_analysis.simulation import lmoma

import escher

## Define helper funcitons and IDs of knocked-out reactions

In [2]:
models_path = Path("../data/models")
simulation_path = Path("../data/simulation_results")
data_path = Path("../data")

In [3]:
def get_knockouts():
    return [
        {"gene": "fbaA", "id": "FBA"},
        {"gene": "fbaB", "id": "FBA"},
        {"gene": "fbp", "id": "FBP"},
        {"gene": "gnd", "id": "GND"},
        {"gene": "pfkA", "id": "PFK"},
        {"gene": "pfkB", "id": "PFK"},
        {"gene": "pgi", "id": "PGI"},
        {"gene": "pgl", "id": "PGL"},
        {"gene": "ppsA", "id": "PPS"},
        {"gene": "pts", "id": "GLCptspp"},
        {"gene": "pykA", "id": "PYK"},
        {"gene": "pykF", "id": "PYK"},
        {"gene": "rpe", "id": "RPE"},
        {"gene": "rpiA", "id": "RPI"},
        {"gene": "rpiB", "id": "RPI"},
        {"gene": "sdhCD", "id": "SUCDi"},
        {"gene": "sucAB", "id": "AKGDH"},
        {"gene": "talA", "id": "TALA"},
        {"gene": "tktA", "id": "TKT1"},
        {"gene": "tktB", "id": "TKT1"},
        {"gene": "tpi", "id": "TPI"},
        {"gene": "zwf", "id": "G6PDH2r"},
        {"gene": "gpmA", "id": "PGM"},
    ]

In [4]:
def show_escher(result):
    escher.Builder(
    map_name="iJO1366.Central metabolism",
    reaction_data=result.fluxes.to_dict(),
).display_in_browser()

In [5]:
def prepare_dataframe(
    solution, sample="WT", author="iML1515", glucose_flux_id="EX_glc__D_e"
):
    # Prepare basic DataFrame
    df = pd.DataFrame(solution.fluxes)
    df.index = df.index.rename("ID")
    df = df.reset_index()
    df = df.assign(sample_id=sample)

    # rename some columns to be more compatible with other simulations
    df = df.assign(author=author)
    df = df.assign(BiGG_ID=df.ID)
    df = df.rename({"fluxes": "flux"}, axis=1)

    # calculate normalized fluxes
    glucose_uptake = -1 * df[df["ID"] == glucose_flux_id]["flux"].values[0]
    df = df.assign(normalized_flux=lambda x: x.flux * 100 / glucose_uptake)
    return df

In [6]:
def simulate_knockouts(
    model,
    ref_flux,
    knockouts=get_knockouts(),
    data_so_far=None,
    save_path=None,
    author="iML1515",
    glucose_flux_id="EX_glc__D_e",
):
    # ref_flux is result.fluxes, save_path should be path to specific subfolder like COBRA/iML1515,
    # prepare dataframe
    if data_so_far is not None:
        all_data = data_so_far
    else:
        all_data = pd.DataFrame()

    # calculate knockouts
    for reaction in knockouts:
        with model:
            ko_gene = reaction["gene"]
            print(f"Working on gene {ko_gene}")
            try:
                # Simulate knockout, it may not grow at all
                model.reactions.get_by_id(reaction["id"]).knock_out()
                lmoma_result = lmoma(model, reference=ref_flux)
                df = prepare_dataframe(
                    lmoma_result,
                    sample=ko_gene,
                    author=author,
                    glucose_flux_id=glucose_flux_id,
                )
                if save_path is not None:
                    df.to_csv(save_path / f"delta_{ko_gene}.csv")
                all_data = pd.concat([all_data, df], sort=False)
            except:
                print(f"Unable to grow {ko_gene}!")
      
    return all_data

### Load experimental data

In [7]:
long_df = pd.read_csv(data_path / "datasets" / "long2019_tidy.csv")
long_df["sample_id"] = long_df.Genotype
long_df = long_df.assign(author="Long").rename(
            {
                "Measurement_ID": "BiGG_ID",
                "Original_Value": "normalized_flux",
                "Value": "flux",
                "Original_ID": "ID",
            }, axis=1,)
long_df = long_df[long_df["Measurement_Type"] == "flux"]
long_df = long_df[~long_df["BiGG_ID"].isna()]
# the same reaction as rpe
long_df = long_df[long_df["sample_id"] != "sgcE"]
long_df = long_df[["BiGG_ID", "ID", "flux", "author", "sample_id", "normalized_flux"]]
exp_results = long_df

In [8]:
exp_data = exp_results.query("sample_id == 'WT'")

In [9]:
exp_data.head()

,BiGG_ID,ID,flux,author,sample_id,normalized_flux
21,GLCptspp,Gluc.Ext + PEP -> G6P + Pyr,8.58000,Long,WT,100.0
22,PGI,G6P <=> F6P (net),5.93736,Long,WT,69.2
23,PFK,F6P + ATP -> FBP,8.75160,Long,WT,102.0
24,FBP,FBP -> F6P,1.81038,Long,WT,21.1
25,FBA,FBP <=> DHAP + GAP (net),6.94122,Long,WT,80.9


## Simulate iML1515

In [10]:
model = cobra.io.load_json_model(str(models_path / "original_files" / "iML1515.json"))

In [11]:
iml1515 = model.copy()

In [15]:
# Minimize glucose consumption from medium - equiv to max exchange. 
#model.objective = "EX_glc__D_e"
#model.objective_direction = "max"
# make glucose uptake to be through Pts
model.reactions.GLCptspp.bounds = (0.0,1000)

In [16]:
# calculate and write WT information
with model:
    # Match glucose uptake rate (only for WT)
    model.reactions.GLCptspp.bounds = (8.57,8.59)
    batch_result = pfba(model)
    df = prepare_dataframe(batch_result, author="iML1515", sample="WT", glucose_flux_id="EX_glc__D_e")

df.to_csv(simulation_path / "COBRA" / "iML1515" / "batch_knockouts" / "WT.csv")

In [17]:
res = simulate_knockouts(model = model, ref_flux=batch_result.fluxes, data_so_far=df, author="iML1515", glucose_flux_id="EX_glc__D_e")
res.to_csv(simulation_path / "COBRA" / "iML1515" / "batch_knockouts" / "knockouts_all.csv")

Working on gene fbaA
Working on gene fbaB
Working on gene fbp
Working on gene gnd
Working on gene pfkA
Working on gene pfkB
Working on gene pgi
Working on gene pgl
Working on gene ppsA
Working on gene pts
Working on gene pykA
Working on gene pykF
Working on gene rpe
Working on gene rpiA
Working on gene rpiB
Working on gene sdhCD
Working on gene sucAB
Working on gene talA
Working on gene tktA
Working on gene tktB
Working on gene tpi
Working on gene zwf
Working on gene gpmA


### Constrain iML1515 with experimental flux data

In [18]:
reversed_fluxes = ["RPI", "PGM", "PGK", "SUCOAS", "TKT1",
                   "ASPTA", "ALATA_L", "VALTA", "ILETA", "PHETA1", "TYRTA", "TRPTA",
                   "EX_atp_e", "EX_ac_e", "EX_co2_e", "EX_o2_e", "EX_nh4_e", "EX_so4_e"]

common_fluxes = set(batch_result.fluxes.index).intersection(set(exp_data.BiGG_ID.unique()))
remove_ids = set(["GLNS", "ASPTA", "EX_co2_e", "EX_nh4_e", "TKT2", "FUM", "GLUSy", "PSERT", "GHMT2r", "MTHFD", "EX_ac_e", "EX_o2_e", "MGSA", "ICDHyr", "GLYCL"])

common_fluxes = {f for f in common_fluxes if f not in remove_ids}

In [19]:
common_fluxes

{'ACONTa',
 'AKGDH',
 'ALATA_L',
 'ARGSL',
 'ASNS2',
 'CS',
 'DAPDC',
 'DAPE',
 'EDA',
 'EDD',
 'EX_so4_e',
 'FBA',
 'FBP',
 'G6PDH2r',
 'GAPD',
 'GLCptspp',
 'GLU5K',
 'GND',
 'HISTD',
 'ICL',
 'ILETA',
 'IPPS',
 'LGTHL',
 'MALS',
 'MDH',
 'ME1',
 'ME2',
 'METS',
 'MTHFR2',
 'NADTRHD',
 'PDH',
 'PFK',
 'PGI',
 'PGM',
 'PHETA1',
 'PPC',
 'PPCK',
 'PTAr',
 'PYK',
 'RPE',
 'RPI',
 'SERAT',
 'SERD_L',
 'SUCDi',
 'SUCOAS',
 'TALA',
 'THRA',
 'THRS',
 'TKT1',
 'TPI',
 'TYRTA',
 'VALTA'}

In [20]:
[f for f in common_fluxes if f in reversed_fluxes]

['PHETA1',
 'VALTA',
 'EX_so4_e',
 'SUCOAS',
 'PGM',
 'ALATA_L',
 'TYRTA',
 'ILETA',
 'TKT1',
 'RPI']

In [21]:
[f for f in remove_ids if f in reversed_fluxes]

['EX_o2_e', 'EX_nh4_e', 'ASPTA', 'EX_co2_e', 'EX_ac_e']

In [22]:
"TKT1" in reversed_fluxes

True

In [23]:
df = exp_data.query("BiGG_ID in @common_fluxes")
exp_model = model.copy()

for row in df.itertuples():
        #print(f"Adding constraint to flux {row.BiGG_ID}")
        #with exp_model:
        r = exp_model.reactions.get_by_id(row.BiGG_ID)
        flux = row.flux
        if row.BiGG_ID in reversed_fluxes:
            flux = -flux
        r.bounds = (flux - 0.1, flux + 0.1)
        #pfba(exp_model)

In [24]:
# exp_data.query("BiGG_ID in @remove_ids").set_index("BiGG_ID").join(exp_res.fluxes.loc[remove_ids])

In [26]:
# Calculate WT
exp_batch_result = pfba(exp_model)
df = prepare_dataframe(exp_batch_result, sample="WT", author="Exp_iML1515", glucose_flux_id="EX_glc__D_e" )
df.to_csv(simulation_path / "COBRA" / "Exp_iML1515" / "batch_knockouts" / "WT.csv")

In [27]:
res = simulate_knockouts(model = model, ref_flux=exp_batch_result.fluxes, data_so_far=df, author="Exp_iML1515", glucose_flux_id="EX_glc__D_e")
res.to_csv(simulation_path / "COBRA" / "Exp_iML1515" / "batch_knockouts" / "knockouts_all.csv")

Working on gene fbaA
Working on gene fbaB
Working on gene fbp
Working on gene gnd
Working on gene pfkA
Working on gene pfkB
Working on gene pgi
Working on gene pgl
Working on gene ppsA
Working on gene pts
Working on gene pykA
Working on gene pykF
Working on gene rpe
Working on gene rpiA
Working on gene rpiB
Working on gene sdhCD
Working on gene sucAB
Working on gene talA
Working on gene tktA
Working on gene tktB
Working on gene tpi
Working on gene zwf
Working on gene gpmA


## Simulate EColiCore2 model

In [28]:
model = cobra.io.read_sbml_model(str(models_path / "original_files" / "EColiCore2_compressed_bigg_names.sbml"))

Adding exchange reaction EX_Biomass for boundary metabolite: Biomass
Adding exchange reaction EX_4CRSOL_ex for boundary metabolite: 4CRSOL_ex
Adding exchange reaction EX_5DRIB_ex for boundary metabolite: 5DRIB_ex
Adding exchange reaction EX_ac_ex for boundary metabolite: ac_ex
Adding exchange reaction EX_adp_c for boundary metabolite: adp_c
Adding exchange reaction EX_AMOB_ex for boundary metabolite: AMOB_ex
Adding exchange reaction EX_ca2_ex for boundary metabolite: ca2_ex
Adding exchange reaction EX_cl_ex for boundary metabolite: cl_ex
Adding exchange reaction EX_co2_ex for boundary metabolite: co2_ex
Adding exchange reaction EX_coa_c for boundary metabolite: coa_c
Adding exchange reaction EX_cobalt2_ex for boundary metabolite: cobalt2_ex
Adding exchange reaction EX_cu2_ex for boundary metabolite: cu2_ex
Adding exchange reaction EX_etoh_ex for boundary metabolite: etoh_ex
Adding exchange reaction EX_fe2_ex for boundary metabolite: fe2_ex
Adding exchange reaction EX_fe3_ex for boundar

In [29]:
ecc = model.copy()

In [30]:
# recreate medium as in iML1515
med_ids = [reaction + 'x' for reaction in iml1515.medium]

for reaction in model.medium:
    if reaction not in med_ids:
        model.reactions.get_by_any(reaction)[0].bounds = (0,1000)
    else:
        model.reactions.get_by_any(reaction)[0].bounds = (-1000,1000)

# set glucose level to match iML1515
model.exchanges.EX_glc__D_ex.bounds = (-10.0,1000)

In [32]:
# calculate and write WT information
with model:
    # Match glucose uptake rate (only for WT)
    model.reactions.GLCptspp.bounds = (8.5,8.59)
    batch_result = pfba(model)
    df = prepare_dataframe(batch_result, author="ECC2", glucose_flux_id="EX_glc__D_ex")
    df.to_csv(simulation_path / "COBRA" / "ECC2" / "batch_knockouts" / "WT.csv")

In [33]:
res = simulate_knockouts(model = model, ref_flux=batch_result.fluxes, data_so_far=df, author="ECC2", glucose_flux_id="EX_glc__D_ex")
res.to_csv(simulation_path / "COBRA" / "ECC2" / "batch_knockouts" /  "knockouts_all.csv")

Working on gene fbaA
Working on gene fbaB
Working on gene fbp
Working on gene gnd
Working on gene pfkA
Working on gene pfkB
Working on gene pgi
Working on gene pgl
Working on gene ppsA
Working on gene pts
Working on gene pykA
Working on gene pykF
Working on gene rpe
Working on gene rpiA
Working on gene rpiB
Working on gene sdhCD
Working on gene sucAB
Working on gene talA
Working on gene tktA
Working on gene tktB
Working on gene tpi
Working on gene zwf
Working on gene gpmA


### Constrain ECC2 by experimentally measured fluxes

#### Identify which fluxes could be constrainted

In [35]:
reversed_fluxes = ["RPI", "TKT1",
                   "ASPTA", "ALATA_L", "VALTA", "ILETA", "PHETA1", "TYRTA", "TRPTA",
                   "EX_atp_e", "EX_ac_e", "EX_co2_e", "EX_o2_e", "EX_nh4_e", "EX_so4_e"]

common_fluxes = set(batch_result.fluxes.index).intersection(set(exp_data.BiGG_ID.unique()))
remove_ids = set(["GLNS", "ASPTA", "EX_co2_e", "EX_nh4_e", "TKT2", "FUM", "GLUSy", "PSERT", "GHMT2r", "MTHFD", "EX_ac_e", "EX_o2_e", "MGSA", "GLYCL", "PPC", "PTAr"])

common_fluxes = {f for f in common_fluxes if f not in remove_ids}

In [36]:
common_fluxes

{'ACONTa',
 'AKGDH',
 'CS',
 'EDA',
 'EDD',
 'FBA',
 'FBP',
 'G6PDH2r',
 'GAPD',
 'GLCptspp',
 'GND',
 'ICDHyr',
 'ICL',
 'MALS',
 'MDH',
 'ME1',
 'ME2',
 'NADTRHD',
 'PDH',
 'PFK',
 'PGI',
 'PGM',
 'PPCK',
 'PYK',
 'RPE',
 'RPI',
 'SUCDi',
 'SUCOAS',
 'TALA',
 'TKT1',
 'TPI'}

In [37]:
df = exp_data.query("BiGG_ID in @common_fluxes")
exp_model = model.copy()

for row in df.itertuples():
        #print(f"Setting bounds for reaction {row.BiGG_ID}")
        r = exp_model.reactions.get_by_id(row.BiGG_ID)
        flux = row.flux
        if row.BiGG_ID in reversed_fluxes:
            flux = -flux
        r.bounds = (flux - 0.1, flux + 0.1)
        #pfba(exp_model)

In [38]:
#exp_data.query("BiGG_ID in @remove_ids").set_index("BiGG_ID").join(exp_batch_result.fluxes.loc[remove_ids])

In [39]:
# Calculate WT
exp_batch_result = pfba(exp_model)
df = prepare_dataframe(exp_batch_result, sample="WT", author="Exp_ECC2", glucose_flux_id="EX_glc__D_ex" )
df.to_csv(simulation_path / "COBRA" / "Exp_ECC2" / "batch_knockouts" / "WT.csv")

In [40]:
res = simulate_knockouts(model = model, ref_flux=exp_batch_result.fluxes, data_so_far=df, author="Exp_ECC2", glucose_flux_id="EX_glc__D_ex")
res.to_csv(simulation_path / "COBRA" / "Exp_ECC2" / "batch_knockouts" / "knockouts_all.csv")

Working on gene fbaA
Working on gene fbaB
Working on gene fbp
Working on gene gnd
Working on gene pfkA
Working on gene pfkB
Working on gene pgi
Working on gene pgl
Working on gene ppsA
Working on gene pts
Working on gene pykA
Working on gene pykF
Working on gene rpe
Working on gene rpiA
Working on gene rpiB
Working on gene sdhCD
Working on gene sucAB
Working on gene talA
Working on gene tktA
Working on gene tktB
Working on gene tpi
Working on gene zwf
Working on gene gpmA
